In [ ]:
#| default_exp installing

In [ ]:
#| export
import subprocess
import shutil
import json

In [ ]:
#| export

class ArduinoNano33BLE_Installer:

    def compile(self, model_path : str):
        "Compiles arduino sketch in docker, using model file at `model_path` in local filesystem"
        shutil.copyfile(model_path, "arduino/template/person_detect_model_data.cpp")
        subprocess.run(['docker build -t nano33ble arduino'], shell=True)

    def upload(self, port:str):
        "Uploads compiled sketch in docker"
        cmd = f"docker run --privileged nano33ble upload -p {port} --fqbn arduino:mbed_nano:nano33ble template"
        subprocess.run([cmd], shell=True)

    def list_ports(self):
        "Returns a list of ports of connected devices"
        ports = []
        cmd_output = subprocess.run(['docker run --privileged nano33ble board list --format=json'], shell=True,capture_output=True).stdout
        result = json.loads(cmd_output.decode("ascii"))
        for row in result:
            ports.append(row["port"]["address"])
        return ports

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
show_doc(ArduinoNano33BLE_Installer.compile)

---

### ArduinoNano33BLE_Installer.compile

>      ArduinoNano33BLE_Installer.compile (model_path:str)

Compiles arduino sketch in docker, using model file at `model_path`

In [ ]:
show_doc(ArduinoNano33BLE_Installer.compile)